Esse código extrai as informações de arquivo HDF5 do radar do Cemaden. Esses arquivos são gerados por [Thiago Biscaro](https://github.com/tsbiscaro) e os dados originais são do [Cemaden](http://www2.cemaden.gov.br/mapainterativo/).


In [1]:
# instalando o py_art
!pip install -q arm_pyart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 13.1 MB/s eta 0:00:00


In [2]:
# importando bibliotecas
import pyart
import csv


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
# monta drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Insira o caminho do diretório onde esta o código:

In [4]:
# diretório
dir = '/content/drive/MyDrive/PYHTON/00_GITHUB/6_RADAR/4_Extrai_info_radar_HDF5/'

In [5]:
#-----------------------------------------------------------------------------------
#        Obtem as informações do radar a partir do dado bruto (HDF5)
#-----------------------------------------------------------------------------------
def get_info_radar(radar):

    """
    Obtem as informações dos dados do radar

    Parâmetros de entrada:
                         radar (object): objeto do pyart

    Parâmetros de saída:
                         dic (dicionário): informações do radar, como:
                           date_start(list): ano, mes, dia hora, minuto inicial do scan do radar
                           date_end(list): ano, mes, dia hora, minuto final do scan do radar
                           lon_radar(float): longitude do radar em graus
                           lat_radar(float): latitude do radar em graus
                           alt_radar(float): altura do radar em metros
                           radar_frequency(float): frequência do radar em Hz
                           radar_beam_width(float): largura do feixe em graus
                           nsweeps(int): quantidade de elevações
                           sweeps(list): ângulos de elevação em graus
                           nrays(list): quantidade de azimutes para cada elevação
                           ngates(int): quantidade de bins
                           radial_resolution(float): resolução radial do radar em metros
                           ray_angle_res(float): resolução azimutal em graus
                           lonmin(float): longitude mínima da matriz de dados
                           lonmax(float): longitude máxima da matriz de dados
                           latmin(float): latitude mínima da matriz de dados
                           latmax(float): latitude máxima da matriz de dados
                           variables(list): variáveis disponíveis

    Exemplo: dic = get_info_radar(radar)
    """

    # importa biblioteca de datas
    from netCDF4 import num2date

    # campos disponíveis
    variaveis = radar.fields.keys()

    # latitude e longitude do radar
    lon_radar, lat_radar, alt_radar = radar.longitude['data'][0], radar.latitude['data'][0], radar.altitude['data'][0]

    # informações sobre o tempo
    data_ini = num2date(radar.time['data'][0], radar.time['units'] )
    data_end = num2date(radar.time['data'][-1], radar.time['units'] )

    # extrai o ano, mes dia, hora e minuto inicial da imagem
    anoi, mesi, diai, hori, mini, segi = str(data_ini.year), str(data_ini.month).zfill(2), str(data_ini.day).zfill(2), str(data_ini.hour).zfill(2), str(data_ini.minute).zfill(2), str(data_ini.second).zfill(2)
    anof, mesf, diaf, horf, minf, segf = str(data_end.year), str(data_end.month).zfill(2), str(data_end.day).zfill(2), str(data_end.hour).zfill(2), str(data_end.minute).zfill(2), str(data_end.second).zfill(2)

    # frequência do radar
    radar_frequency = radar.instrument_parameters['frequency']['data'][0]

    # largura do feixe
    radar_beam_width = radar.instrument_parameters['radar_beam_width_h']['data'][0]

    # quantidade TOTAL de bins do feixe
    ngates = radar.ngates

    # quantidade de elevações
    nsweeps = radar.nsweeps

    # quantidade de azimutes por elevação
    total_azimuths = radar.azimuth['data'].shape[0]
    index_azimuths = radar.sweep_start_ray_index['data'].tolist()
    index_azimuths.append(total_azimuths)
    qte_azim = [index_azimuths[i+1] - index_azimuths[i] for i in range(nsweeps)]

    # resolução radial
    radial_resolution = radar.range['meters_between_gates']

    # informação apenas da resolução ângulo azimutal
    ray_angle_res = radar.ray_angle_res['data'][0]

    # ângulos de elevação
    sweeps =  radar.fixed_angle['data']

    # definindo a extensão dos dados em termos de latitude/longitude
    lats = radar.gate_latitude
    lons = radar.gate_longitude
    lonmin = lons['data'].min()
    lonmax = lons['data'].max()
    latmin = lats['data'].min()
    latmax = lats['data'].max()

    # coloca as variáveis num dicionário
    dic = {
            'date_start(year,month,day,hour,minute)': [anoi, mesi, diai, hori, mini],
            'date_end(year,month,day,hour,minute)': [anof, mesf, diaf, horf, minf],
            'lon_radar(degree)': lon_radar,
            'lat_radar(degree)': lat_radar,
            'alt_radar(meters)': alt_radar,
            'radar_frequency(Hz)': radar_frequency,
            'radar_beam_width(degree)': radar_beam_width,
            'nsweeps(#)': nsweeps,
            'sweeps(degree)': list(sweeps),
            'nrays(#)': qte_azim,
            'ngates(#)': ngates,
            'radial_resolution(meters)': radial_resolution,
            'ray_angle_res(degree)': ray_angle_res,
            'lonmin(degree)': lonmin,
            'lonmax(degree)': lonmax,
            'latmin(degree)': latmin,
            'latmax(degree)': latmax,
            'variables': list(variaveis)

           }

    return dic

In [6]:
#-----------------------------#
#         SAO_FRANCISCO_MG
#-----------------------------#
# nome do radar
name_radar = 'SAO_FRANCISCO_MG'

# arquivo
file_ALMENARA_MG = f'{dir}input/SF1-20211223003004.HDF5'

# leitura
radar = pyart.aux_io.read_gamic(file_ALMENARA_MG)

# extrai informações
dic = get_info_radar(radar)

# salva
with open(f'{dir}info_radar_{name_radar}.csv', 'w') as f:
    for key in dic.keys():
        f.write("%s= %s\n"%(key, dic[key]))

In [7]:
dic

{'date_start(year,month,day,hour,minute)': ['2021', '12', '23', '00', '30'],
 'date_end(year,month,day,hour,minute)': ['2021', '12', '23', '00', '37'],
 'lon_radar(degree)': -44.69588,
 'lat_radar(degree)': -16.00889,
 'alt_radar(meters)': 730.0,
 'radar_frequency(Hz)': 2857888000.0,
 'radar_beam_width(degree)': 1.02,
 'nsweeps(#)': 13,
 'sweeps(degree)': [0.3,
  0.9,
  1.6,
  2.5,
  3.4,
  4.6,
  5.9,
  7.4,
  9.2,
  11.4,
  13.8,
  16.7,
  20.0],
 'nrays(#)': [360, 360, 360, 360, 360, 360, 360, 360, 360, 360, 360, 360, 360],
 'ngates(#)': 1000,
 'radial_resolution(meters)': 250.0,
 'ray_angle_res(degree)': 1.0,
 'lonmin(degree)': -47.031532,
 'lonmax(degree)': -42.36022,
 'latmin(degree)': -18.253836,
 'latmax(degree)': -13.763937,
 'variables': ['corrected_reflectivity',
  'reflectivity',
  'corrected_velocity',
  'spectrum_width',
  'corrected_differential_reflectivity',
  'corrected_differential_phase',
  'differential_phase',
  'specific_differential_phase',
  'cross_correlation_